<a href="https://colab.research.google.com/github/rohitratnakumar/llmDemystifiedProject/blob/main/LLMDemystifiedCode_pynbv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
#!pip install gradio
!pip install langchain-community
!pip install unstructured

#import ntltk
#nltk.download("punkt")
#!pip install --upgrade nltk

#!pip install openai
#!pip install chromadb

#!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.5/150.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 45.3 MB/s eta 0:00:00
   

In [76]:
#setup variables
CHROMA_PATH = 'Chroma'
SOURCE_PATH = 'Sources'
TEST_PATH = 'Testcases'

if not os.path.exists(CHROMA_PATH):
    os.makedirs(CHROMA_PATH)

if not os.path.exists(SOURCE_PATH):
    os.makedirs(SOURCE_PATH)

if not os.path.exists(TEST_PATH):
    os.makedirs(TEST_PATH)

from google.colab import userdata
OPENAPI_KEY=userdata.get('OPENAPI_KEY')
api_key = OPENAPI_KEY



In [77]:
import gradio as gr
import os
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import shutil

loaders = None
uploadedSourceFiles = []
uploadedTestFiles = []

def uploadSourcesFile(files):
    file_paths = [file.name for file in files]
    for file in files:
        shutil.copy(file.name, SOURCE_PATH)
        uploadedSourceFiles.append(file.name)
    return file_paths

def uploadTestsFile(files):
    file_paths = [file.name for file in files]
    for file in files:
        shutil.copy(file.name, TEST_PATH)
        uploadedTestFiles.append(file.name)
    return file_paths

def processCode(codeText):

    tmp = []
    dirs = os.listdir(SOURCE_PATH)
    for f in dirs:
        tmp.append(f)

    dirs2 = os.listdir(TEST_PATH)
    for f2 in dirs2:
        tmp.append(f2)

    return ','.join(tmp)

with gr.Blocks() as demo:

    #step1: sources
    sourcesFileOutput = gr.File()
    uploadSourcesButton = gr.UploadButton("Step 1: Upload Source Files", file_types=["code"], file_count="multiple")
    uploadSourcesButton.upload(uploadSourcesFile, uploadSourcesButton, sourcesFileOutput)

    #step2: sample unit test files
    testsFileOutput = gr.File()
    uploadTestsButton = gr.UploadButton("Step 2: Upload Test Files", file_types=["code"], file_count="multiple")
    uploadTestsButton.upload(uploadTestsFile, uploadTestsButton, testsFileOutput)


    #Input and Output
    inputCodeDiffBox = gr.Code(label = "Step 3: Input Git Code Diff")
    outputTestcasesBox = gr.Code(label = "Output: Generated Testcases")
    processButton = gr.Button("Step 4: Generate TestCases")
    processButton.click(processCode, inputCodeDiffBox, outputTestcasesBox)



demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d5448c459f88fe7161.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
def load_documents():
  loader = DirectoryLoader('./', glob='**/*.java')
  documents = loader.load()
  return documents

In [ ]:
def generate_data_store():
  documents = load_documents()
  chunks = split_text(documents)
  save_to_chroma(chunks)


In [ ]:
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[0]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [ ]:
def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    #if os.path.exists(CHROMA_PATH):
    #    shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, OpenAIEmbeddings(openai_api_key=OPENAPI_KEY), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [ ]:
generate_data_store()

Split 1 documents into 29 chunks.
package org.jsoup.parser; import org.jsoup.Connection; import org.jsoup.helper.Validate; import org.jsoup.nodes.Document; import org.jsoup.nodes.Element; import org.jsoup.nodes.Node; import org.jsoup.select.Evaluator; import org.jsoup.select.NodeVisitor; import org.jsoup.select.QueryParser; import
{'source': 'StreamParser.java', 'start_index': 0}
Saved 29 chunks to ./Chroma.


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [ ]:
import argparse
from dataclasses import dataclass
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [ ]:
query_text = " Write 3 java unit tests for parse function"


In [ ]:
embedding_function = OpenAIEmbeddings(openai_api_key=OPENAPI_KEY)
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
results = db.similarity_search_with_relevance_scores(query_text, k=3)
print(results)

[(Document(metadata={'source': 'StreamParser.java', 'start_index': 218}, page_content='import org.jsoup.select.NodeVisitor; import org.jsoup.select.QueryParser; import org.jspecify.annotations.Nullable; import java.io.Closeable; import java.io.IOException; import java.io.Reader; import java.io.StringReader; import java.io.UncheckedIOException; import java.util.Iterator; import'), 0.6024511697367243), (Document(metadata={'source': 'StreamParser.java', 'start_index': 2291}, page_content="The StreamParser interface is currently in beta and may change in subsequent releases. Feedback on the feature and how you're using it is very welcome via the jsoup discussions."), 0.5926895174983706), (Document(metadata={'source': 'StreamParser.java', 'start_index': 0}, page_content='package org.jsoup.parser; import org.jsoup.Connection; import org.jsoup.helper.Validate; import org.jsoup.nodes.Document; import org.jsoup.nodes.Element; import org.jsoup.nodes.Node; import org.jsoup.select.Evaluator; impor

In [ ]:
# Augument the context
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt)

Human: 
Answer the question based only on the following context:

import org.jsoup.select.NodeVisitor; import org.jsoup.select.QueryParser; import org.jspecify.annotations.Nullable; import java.io.Closeable; import java.io.IOException; import java.io.Reader; import java.io.StringReader; import java.io.UncheckedIOException; import java.util.Iterator; import

---

The StreamParser interface is currently in beta and may change in subsequent releases. Feedback on the feature and how you're using it is very welcome via the jsoup discussions.

---

package org.jsoup.parser; import org.jsoup.Connection; import org.jsoup.helper.Validate; import org.jsoup.nodes.Document; import org.jsoup.nodes.Element; import org.jsoup.nodes.Node; import org.jsoup.select.Evaluator; import org.jsoup.select.NodeVisitor; import org.jsoup.select.QueryParser; import

---

Answer the question based on the above context:  Write 3 java unit tests for parse function



In [ ]:
model = ChatOpenAI(openai_api_key=OPENAPI_KEY)
response_text = model.predict(prompt)

sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response: 1. Test that the parse function correctly parses a valid HTML document and returns a non-null Document object.
2. Test that the parse function throws an IOException when attempting to parse an invalid or malformed HTML document.
3. Test that the parse function correctly handles HTML documents with special characters and returns a valid Document object.
Sources: ['StreamParser.java', 'StreamParser.java', 'StreamParser.java']
